In [35]:
%matplotlib inline
import scipy.io
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from wide_resnet import WideResNet
from MyModel import MyModel




os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
# load model and weights
img_size = 224
model64 = MyModel(img_size)()
model64.load_weights(os.path.join("models", "WRN_16_8.h5"))

#img_size32 = 32
#model32 = WideResNet(img_size32, depth=16, k=8)()
#model32.load_weights(os.path.join("../../dataset/checkpoints", "model32.hdf5"))


In [ ]:
db = "wiki"
data = scipy.io.loadmat(os.path.join("../../dataset/", "{}32.mat".format(db)))
test_images = data["image"]
test_genders = data["gender"][0]
test_ages = data["age"][0]

cols, rows = 4, 3
img_num = cols * rows
path_root = "data/{}_crop/".format(db)
img_ids = np.random.choice(len(test_ages), img_num)
sub_test_images = test_images[img_ids,:,:,:]

# predict
results = model32.predict(sub_test_images)
predicted_genders = results[0]
ages = np.arange(0, 101).reshape(101, 1)
predicted_ages = results[1].dot(ages).flatten()

for i in range(img_num):
    plt.subplot(rows, cols, i + 1)
    plt.imshow(cv2.cvtColor(sub_test_images[i,:,:,:], cv2.COLOR_BGR2RGB))
    plt.title("{}, {}".format(int(predicted_ages[i]),
                              "F" if predicted_genders[i][0]>0.5 else "M"))
    plt.axis('off')
plt.savefig("result.png")

In [ ]:
from os import listdir
from os.path import isfile, join
import face_recognition
mypath='test/'
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
#images32 = np.empty(len(onlyfiles), dtype=object)
#faces32 = np.empty((len(onlyfiles), img_size32, img_size32, 3))
images64 = np.empty(len(onlyfiles), dtype=object)
faces64 = np.empty((len(onlyfiles), 224, 224, 3))
for n in range(0, len(onlyfiles)):
   # images32[n] = cv2.imread( join(mypath,onlyfiles[n]))
    #images32[n] = cv2.cvtColor(images32[n],cv2.COLOR_BGR2RGB)
    #faces32[n, : , : , :] = cv2.resize(images32[n][35:-35,35:-35,:], (img_size32, img_size32))
    image = face_recognition.load_image_file(join(mypath,onlyfiles[n]))
    face_locations = face_recognition.face_locations(image)[0]
    #print(face_locations)
    #print(face_locations[0],face_locations[2])
    #print(face_locations[3],face_locations[1])
    #print(image.shape)
    images64[n] = image[face_locations[0]:face_locations[2],face_locations[3]:face_locations[1],:]
    faces64[n, : , : , :] = cv2.resize(images64[n], (224, 224))
    
#results32 = model32.predict(faces32)
results64 = model64.predict(faces64)
#predicted_genders32 = results32[0]
#predicted_genders64 = results64[0]
ages = np.arange(0, 101).reshape(101, 1)
#predicted_ages32 = results32[1].dot(ages).flatten()
predicted_ages64 = results64.dot(ages).flatten()

for n in range(len(onlyfiles)):
    plt.figure()
   # plt.imshow(cv2.resize(images32[n], (img_size64, img_size64)))
    #print("Age " +onlyfiles[n]+ " (32x32): ", int(predicted_ages32[n]))
    #print("Gender " +onlyfiles[n]+ " (32x32): ", "F" if predicted_genders32[n][0] > 0.5 else "M")
    plt.figure()
    plt.imshow(images64[n])
    print("Age " +onlyfiles[n]+ " (64x64): ", predicted_ages64[n])
    #print("Gender " +onlyfiles[n]+ " (64x64): ", "F" if predicted_genders64[n][0] > 0.5 else "M")

In [36]:
from keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

val_generator = train_datagen.flow_from_directory(
    '../../dataset/wiki_crop/new_database/',
    target_size=(img_size, img_size),
    class_mode='categorical',
    subset='validation')


Found 7594 images belonging to 101 classes.


In [37]:

def x_difference(val_generator, X):
    predictions = model64.predict_generator(val_generator)
    ages = np.arange(0, 101).reshape(101, 1)
    predicted_age = predictions.dot(ages).flatten()
    counter = 0

    for i in range(len(val_generator.classes)):
        real_label = val_generator.classes[i]
        if abs(real_label - predicted_age[i]) < X:
            counter += 1
            
    return counter / len(val_generator.classes)

print(x_difference(val_generator,5))


0.39952594153278903
